In [38]:
import pandas as pd
import requests

In [39]:
#---------------------------------------------------
#
# Config
#
#---------------------------------------------------

# This file should be automatically generated from "1. Gaze Projections and AOIs Mapping"
GAZE_DATA_WITH_AOIS_FILE = "data/gazeDataWithAOIs.csv"

In [40]:
#----------------------------------------------------------------------------------------
#
#
# 1. Oculomotor event detection (i.e., Fixations and saccades)
# 
#
#----------------------------------------------------------------------------------------

In [41]:
# Read gaze data with AOIs using pandas library
data = pd.read_csv(GAZE_DATA_WITH_AOIS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [42]:
# Preview data
data.head()

Row Respondent  Age Gender    Group Calibration  Timestamp  EventSource  \
0    1        P01    0  OTHER  Default   Excellent   112.2004          1.0   
1    2        P01    0  OTHER  Default   Excellent   113.2910          NaN   
2    3        P01    0  OTHER  Default   Excellent   121.6164          NaN   
3    4        P01    0  OTHER  Default   Excellent   129.9499          NaN   
4    5        P01    0  OTHER  Default   Excellent   138.2936          NaN   

   SlideEvent   StimType  Duration CollectionPhase SourceStimuliName  \
0  StartSlide  TestImage  300000.0  StimuliDisplay            Slide1   
1         NaN        NaN       NaN             NaN            Slide1   
2         NaN        NaN       NaN             NaN            Slide1   
3         NaN        NaN       NaN             NaN            Slide1   
4         NaN        NaN       NaN             NaN            Slide1   

   EventSource.1  ET_GazeLeftx  ET_GazeLefty  ET_GazeRightx  ET_GazeRighty  \
0            NaN           NaN           NaN            NaN            NaN   
1            1.0        1074.0         590.0         1081.0          527.0   
2            1.0        1080.0         579.0         1078.0          541.0   
3            1.0        1078.0         572.0         1082.0          559.0   
4            1.0           NaN           NaN            NaN            NaN   

   ET_PupilLeft  ET_PupilRight  ET_TimeSignal  ET_DistanceLeft  \
0           NaN            NaN            NaN              NaN   
1      3.323868       3.239807        149.999       657.255432   
2           NaN            NaN        158.325       657.347839   
3           NaN            NaN        166.658       657.481262   
4           NaN            NaN        175.002              NaN   

   ET_DistanceRight  ET_CameraLeftX  ET_CameraLeftY  ET_CameraRightX  \
0               NaN             NaN             NaN              NaN   
1        650.518372        0.568510        0.498617         0.419377   
2        650.507507        0.568528        0.498499         0.419368   
3        650.567383        0.568659        0.498408         0.419488   
4               NaN             NaN             NaN              NaN   

   ET_CameraRightY  ET_ValidityLeft  ET_ValidityRight   GazeX  GazeY  Chat  \
0              NaN              NaN               NaN     NaN    NaN     0   
1         0.498766              0.0               0.0  1077.5  558.5     0   
2         0.498608              0.0               0.0  1079.0  560.0     0   
3         0.498481              0.0               0.0  1080.0  565.5     0   
4              NaN              4.0               4.0     NaN    NaN     0   

   CTA_1  Date_Place_1  Date_Place_2  Description_1  Description_2  Fav  Food  \
0      0             0             0              0              0    0     0   
1      0             0             0              0              0    0     0   
2      0             0             0              0              0    0     0   
3      0             0             0              0              0    0     0   
4      0             0             0              0              0    0     0   

   Gen_Info  Ingredients  Instructions  Online  Participants_Overview  \
0         0            0             0       0                      0   
1         0            0             0       0                      0   
2         0            0             0       0                      0   
3         0            0             0       0                      0   
4         0            0             0       0                      0   

   Participant_Bio  Participant_Profile_Picture  Place  Responsabilities  \
0                0                            0      0                 0   
1                0                            0      0                 1   
2                0                            0      0                 1   
3                0                            0      0                 1   
4                0         

In [43]:
#Keep only the gaze data coming from the eye-tracking device
data = data[ data['EventSource.1'] == 1 ]

In [44]:
# Convert object dtypes to more specific types where possible
data = data.infer_objects(copy=False)

#Fill n/a values with -1 (for JSON Compliance in requests) 
data = data.fillna(-1) 



In [ ]:
# Oculomotor event detection (i.e., Fixations and saccades)
# Using customizable I-VT (Velocity Threshold) filter for fixation and saccade detection

dataWithEvents = None

# Iteratate over the participants and the apply I-VT filter on the data of each participant
for participant in data['Respondent'].unique():
    
    # filter in the participant data
    ParticipantData = data[data['Respondent']==participant]
    
    print(f'Applying fixation filter on the data for participant: {participant}')
    config = {
                    "fixationFilter" : "IVT",
                    "screenResolutionWidth" : 1920, 
                    "screenResolutionHeight" : 1080, 
                    "monitorSize" : 24, 
                    "screenDistance" : 60,
                    "gapFill" : True, 
                    "maxGapLength": 75, 
                    "noiseReduction" : False, 
                    "filterType" : "Median", 
                    "windowNoise" : 3,
                    "windowVelocity" : 20, 
                    "velocityThreshold" : 30, 
                    "mergeFixation" : True, 
                    "maxTimeBtwFixation" : 75,
                    "maxAngleBtwFixation" : 0.5, 
                    "discardShortFixation" : True,
                    "minDurationFixation" : 60
    }

    # POST pariticipant gaze data to the (R) eventDetection server
    url = 'http://rserver:6789/eventDetection'
    #url = 'http://127.0.0.1:6789/eventDetection'
    payload = {'config': config, 'data': ParticipantData.to_dict(orient="records")}
    response = requests.post(url, json = payload)
    
    # Create new a dataframe from the json response of the (R) eventDetection server
    participantDataWithEvents = pd.DataFrame.from_records(response.json())
    
    #Assign participantDataWithEvents to dataWithEvents or concate
    if dataWithEvents is None: 
        dataWithEvents = participantDataWithEvents
    else:
        dataWithEvents = pd.concat([dataWithEvents, participantDataWithEvents], axis=0)
        
    print('... Complete')

Applying fixation filter on the data for participant: P01


In [ ]:
# Preview gaze data with event information
dataWithEvents.head()

In [ ]:
# export dataWithEvents as csv
dataWithEvents.to_csv("data/gazeDataWithAOIsAndEvents.csv",  index=False)